| Indicator                                       | Rationale                              | Source     |
| ----------------------------------------------- | -------------------------------------- | ---------- |
| Internet users (% of population)                | Measures baseline digital access       | World Bank |
| Mobile broadband subscriptions (per 100 people) | Reflects mobile internet access        | World Bank |
| Households with a computer (%)                  | Shows infrastructure readiness at home | ITU        |


| Indicator                                | Rationale                             | Source                  |
| ---------------------------------------- | ------------------------------------- | ----------------------- |
| Individuals using the Internet (%)       | Tracks actual digital engagement      | ITU                     |
| Internet banking users (% of population) | Proxy for service digitalisation      | OECD                    |
| Social media usage (% of population)     | Public digital communication adoption | Statista / DataReportal |


| Indicator                                                    | Rationale                          | Source    |
| ------------------------------------------------------------ | ---------------------------------- | --------- |
| ICT graduates as % of total graduates                        | Measures tech-educated workforce   | UNESCO    |
| Digital literacy rate (or adult literacy with ICT component) | Basic ability to use digital tools | ITU / UIS |

| Indicator                                  | Rationale                         | Source                                |
| ------------------------------------------ | --------------------------------- | ------------------------------------- |
| R\&D expenditure (% of GDP)                | National investment in innovation | World Bank                            |
| ICT patent applications per million people | Direct measure of tech innovation | WIPO                                  |
| Number of tech startups per capita         | Entrepreneurial tech activity     | Global Entrepreneurship Monitor (GEM) |


Data Source Overview
Most of the data will be sourced from reputable international organisations that maintain regularly updated, country-level statistics:

- World Bank: https://data.worldbank.org

- ITU (International Telecommunication Union): https://www.itu.int/en/ITU-D/Statistics

- OECD: https://data.oecd.org

- UNESCO Institute for Statistics: http://uis.unesco.org

- WIPO (World Intellectual Property Organization): https://www.wipo.int/ipstats

- Statista/DataReportal: May require API access or manual scraping

Data of most recent years will be prioritised.

Data Format & Processing Plan
- All data will be downloaded in CSV format where possible.

- Country names will be standardized using ISO 3166-1 alpha-3 codes.

- Data will be merged into a single dataframe using the country code as a primary key.

- Missing values will be identified and handled in the next stage using imputation techniques.

<h3> World Bank Data </h3>

* Individuals using the Internet (% of population)
* Fixed broadband subscriptions (per 100 people) 
* Research and development expenditure (% of GDP) 
* Mobile cellular subscriptions (per 100 people) 
* High-technology exports (current US$) 

In [81]:
import pandas as pd

# General-purpose function: loads World Bank data and extracts target column
def load_world_bank_data(file_path, indicator_label="Value", year="2024"):
    df = pd.read_csv(file_path, skiprows=4)
    
    # Use the specified year, or fallback to latest if missing
    if year in df.columns:
        df_filtered = df[['Country Name', year]].rename(columns={year: indicator_label})
    else:
        latest = df.columns[-2]  # fallback to second-last column if year not found
        df_filtered = df[['Country Name', latest]].rename(columns={latest: indicator_label})
    
    return df_filtered.set_index('Country Name')

# === Load all indicator files ===
internet_file = "Datasets/API_IT.NET.USER.ZS_DS2_en_csv_v2_85299.csv"
broadband_file = "Datasets/API_IT.NET.BBND.P2_DS2_en_csv_v2_85398.csv"
rnd_file = "Datasets/API_GB.XPD.RSDV.GD.ZS_DS2_en_csv_v2_85148.csv"
cellular_file = "Datasets/API_IT.CEL.SETS.P2_DS2_en_csv_v2_88708.csv"
exports_file = "Datasets/API_TX.VAL.TECH.CD_DS2_en_csv_v2_92892.csv"

# Load each dataset independently
df_internet = load_world_bank_data(internet_file, "Internet_Users_Pct", "2022")
df_broadband = load_world_bank_data(broadband_file, "Fixed_Broadband_per_100", "2022")
df_rnd = load_world_bank_data(rnd_file, "R&D_Expenditure_Pct_GDP", "2022")
df_cellular = load_world_bank_data(cellular_file, "Mobile_Cellular_per_100", "2022")
df_exports = load_world_bank_data(exports_file, "High_Technology_Exports_USD", "2022")

# === Merge all datasets on Country Name (outer join to retain all countries) ===
combined_df = df_internet.join(df_broadband, how='outer')\
                         .join(df_rnd, how='outer')\
                         .join(df_cellular, how='outer')\
                         .join(df_exports, how='outer')

# === Impute missing values with column means ===
imputed_df = combined_df.fillna(combined_df.mean(numeric_only=True))

# === Output merged dataset ===
print(imputed_df)

# Optional: Save to CSV
imputed_df.to_csv("tech_adoption_imputed_all_countries.csv")


                             Internet_Users_Pct  Fixed_Broadband_per_100  \
Country Name                                                               
Afghanistan                           17.200000                  0.07963   
Africa Eastern and Southern           70.388235                  0.90000   
Africa Western and Central            70.388235                  0.40000   
Albania                               82.600000                 20.69890   
Algeria                               74.800000                 10.35170   
...                                         ...                      ...   
West Bank and Gaza                    88.600000                  7.93673   
World                                 63.700000                 18.42808   
Yemen, Rep.                           70.388235                  1.27124   
Zambia                                32.000000                  0.42895   
Zimbabwe                              37.900000                  1.28584   

           

<h3> ICT Access and Usage by Individuals

In [82]:
import pandas as pd
import pycountry

# Load the CSV
df = pd.read_csv("Datasets/data.csv", skiprows=4, header=None, usecols=[1, 2, 3])

# Assign readable column names
df.columns = ['Country_Code', 'Year', 'ICT_Graduates_Pct']

# Drop rows with missing or malformed country codes
df = df[df['Country_Code'].notna() & (df['Country_Code'].str.len() == 3)]

# Sort so latest year comes first
df = df.sort_values(by=['Country_Code', 'Year'], ascending=[True, False])

# Drop duplicates to keep only the most recent entry per country
df_latest = df.drop_duplicates(subset='Country_Code', keep='first').copy()

# Auto-map ISO-3 country codes to country names using pycountry
def get_country_name(code):
    try:
        return pycountry.countries.get(alpha_3=code).name
    except:
        return None

df_latest['Country Name'] = df_latest['Country_Code'].apply(get_country_name)

# Remove rows where mapping failed
df_latest = df_latest[df_latest['Country Name'].notna()]

# Build final DataFrame
df_final = df_latest[['Country Name', 'ICT_Graduates_Pct']].set_index('Country Name')

# Print or save
print(df_final)
df_final.to_csv("ict_graduates_latest_all_countries.csv")

                         ICT_Graduates_Pct
Country Name                              
Albania                            6.07570
Andorra                           13.63636
United Arab Emirates               8.80694
Argentina                          1.65571
Armenia                            7.22494
...                                    ...
United States                      5.01022
Uzbekistan                         4.82795
Virgin Islands, British            9.61240
Samoa                              0.00000
South Africa                       4.29026

[131 rows x 1 columns]


<h3> Share of Adults Proficient Problem Solving in Technology-Rich Environments

In [83]:
import pandas as pd

# Load the dataset (update path as needed)
file_path = "Datasets/indicator_24_20250508_all.csv"
df = pd.read_csv(file_path)

# Step 1: Filter to the relevant domain
df_filtered = df[df['Domain'] == "Problem solving in technology-rich environments"]

# Step 2: Filter to Level 3 only
df_level3 = df_filtered[df_filtered['Level'] == "Level 3"]

# Step 3: Keep only relevant columns
df_level3_final = df_level3[['Country', 'Value']].rename(
    columns={'Value': 'Problem_Solving_Tech_Level3_Pct'}
).set_index('Country')

# Display result
print(df_level3_final)

# Optional: Save to CSV
df_level3_final.to_csv("problem_solving_level3.csv")


                 Problem_Solving_Tech_Level3_Pct
Country                                         
Australia                                   6.18
Austria                                     4.33
Canada                                      7.12
Chile                                       2.10
Czechia                                     6.61
Denmark                                     6.33
Estonia                                     4.32
Finland                                     8.35
Germany                                     6.80
Greece                                      2.50
Ireland                                     3.14
Israel                                      6.37
Japan                                       8.25
Korea                                       3.60
Lithuania                                   2.06
Netherlands                                 7.28
New Zealand                                10.17
Norway                                      6.07
Poland              

<h3> UNCTAD B2C E-Commerce Index

In [84]:
import pandas as pd

# Load the Excel file
file_path = "Datasets/UNCTAD-B2C.xlsx"
df_b2c = pd.read_excel(file_path, sheet_name="Data")

# Filter only B2C Index (not rank)
df_index = df_b2c[df_b2c['Indicator ID'] == 'UNCTAD.B2C.INDEX']

# Extract 2020 values
df_index_2020 = df_index[['Economy Name', '2020']].rename(
    columns={'Economy Name': 'Country', '2020': 'B2C_Index_2020'}
).set_index('Country')

# View or export
print(df_index_2020)
df_index_2020.to_csv("b2c_index_2020.csv")


                      B2C_Index_2020
Country                             
Afghanistan                     17.1
Angola                          26.0
Albania                         49.5
United Arab Emirates            78.2
Argentina                       50.9
...                              ...
Vietnam                         61.6
Yemen, Rep.                     18.5
South Africa                    56.5
Zambia                          30.0
Zimbabwe                        30.5

[152 rows x 1 columns]


In [85]:
import pandas as pd

# Load the Excel sheet
file_path = "Datasets/DAIforweb.xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")

# Keep only the latest year for each country (2016)
df_latest = df[df['Year'] == 2016].copy()

# Set country as index and rename main columns
df_dai = df_latest[['country', 'Digital Adoption Index', 
                    'DAI Business Sub-index', 
                    'DAI People Sub-index', 
                    'DAI Government Sub-index']]

df_dai = df_dai.set_index('country').rename(columns={
    'Digital Adoption Index': 'DAI_Overall',
    'DAI Business Sub-index': 'DAI_Business',
    'DAI People Sub-index': 'DAI_People',
    'DAI Government Sub-index': 'DAI_Gov'
})

# Output
print(df_dai.to_string())
df_dai.to_csv("digital_adoption_index_2016.csv")


                                DAI_Overall  DAI_Business  DAI_People   DAI_Gov
country                                                                        
Afghanistan                        0.343214      0.341886    0.123343  0.564413
Albania                            0.607904      0.621125    0.463779  0.738807
Algeria                            0.431103      0.498178    0.419688  0.375444
Andorra                            0.640383      0.829273    0.739461  0.352417
Angola                             0.334767      0.408602    0.131608  0.464091
Antigua and Barbuda                0.477325      0.612027    0.565118  0.254831
Argentina                          0.685676      0.690285    0.633734  0.733010
Armenia                            0.621723      0.714228    0.478799  0.672143
Australia                          0.712174      0.772024    0.777300  0.587198
Austria                            0.862427      0.876887    0.865305  0.845087
Azerbaijan                         0.594

In [90]:
import pandas as pd

# Load each dataset
df_core = pd.read_csv("tech_adoption_imputed_all_countries.csv", index_col=0)
df_ict = pd.read_csv("ict_graduates_latest_all_countries.csv", index_col=0)
df_skills = pd.read_csv("problem_solving_level3.csv", index_col=0)
df_b2c = pd.read_csv("b2c_index_2020.csv", index_col=0)
df_dai = pd.read_csv("digital_adoption_index_2016.csv", index_col=0)

# Ensure index is trimmed for whitespace
for df in [df_core, df_ict, df_skills, df_b2c, df_dai]:
    df.index = df.index.str.strip()

# Get sets of country names in each dataset
country_sets = [set(df.index) for df in [df_core, df_ict, df_skills, df_b2c, df_dai]]

# Find intersection
common_countries = set.intersection(*country_sets)

# Print count and list
print(f"Common countries: {len(common_countries)}")
common_countries = list(common_countries)  # convert set to list

df_core = df_core.loc[common_countries]
df_ict = df_ict.loc[common_countries]
df_skills = df_skills.loc[common_countries]
df_b2c = df_b2c.loc[common_countries]
df_dai = df_dai.loc[common_countries]

print(sorted(common_countries))
# Filter all datasets to only those common countries
df_core = df_core.loc[common_countries]
df_ict = df_ict.loc[common_countries]
df_skills = df_skills.loc[common_countries]
df_b2c = df_b2c.loc[common_countries]
df_dai = df_dai.loc[common_countries]

# Merge everything
df_merged = df_core.join([df_ict, df_skills, df_b2c, df_dai])

# Save final combined dataset
df_merged.to_csv("final_tech_adoption_index_data.csv")

# Show result
print(df_merged.to_string())


Common countries: 19
['Australia', 'Austria', 'Canada', 'Chile', 'Denmark', 'Estonia', 'Finland', 'Germany', 'Greece', 'Ireland', 'Israel', 'Lithuania', 'Netherlands', 'New Zealand', 'Norway', 'Poland', 'Slovenia', 'Sweden', 'United States']
               Internet_Users_Pct  Fixed_Broadband_per_100  R&D_Expenditure_Pct_GDP  Mobile_Cellular_per_100  High_Technology_Exports_USD  ICT_Graduates_Pct  Problem_Solving_Tech_Level3_Pct  B2C_Index_2020  DAI_Overall  DAI_Business  DAI_People   DAI_Gov
Country Name                                                                                                                                                                                                                                                  
United States                92.2                  37.4914                 3.586230               110.166472                 1.918761e+11            5.01022                             5.10            91.0     0.747118      0.781266    0.730538  0.